# 四、使用评价指标工具

在训练和测试一个模型时往往需要计算不同的评价指标，如正确率、查准率、查全率、F1值等，具体需要的指标往往和处理的数据集、任务类型有关。HuggingFace提供了统一的评价指标工具，能够将具体的计算过程隐藏，调用者只需提供计算结果，由评价指标工具给出评价指标。

https://huggingface.co/docs/evaluate/index

### 注意：以下代码可能因为网络原因无法正常调用
参考这里 https://blog.csdn.net/misaki_min/article/details/132650725
从 https://github.com/huggingface/evaluate 直接下载evaluate代码

访问上面的github网站，获得下载地址，然后用git命令或者wget命令下载

`git clone https://github.com/huggingface/evaluate.git`

`wget https://github.com/huggingface/evaluate/archive/refs/heads/main.zip`

Evaluate库中的Metric(指标)、Comparison(比较)和Measurement(测量)是三种不同的评估工具，用于评估机器学习模型和数据集。它们之间的关系和区别如下：

1. Metric(指标)：
* 用途：用于评估模型的性能
* 具体含义：指标通过将模型的预测结果与真实标签进行比较来衡量模型的表现
* 示例：准确率、精确率、召回率、F1分数等
* 目的：提供了对模型性能的定量评估，帮助衡量模型在特定任务上的表现
2. Comparison(比较)：
* 用途：用于比较两个模型之间的差异
* 具体含义：比较工具将两个模型的预测结果与真实标签进行对比，计算它们之间的一致性或差异程度
* 示例：一致性指标、相对误差等
* 目的：帮助评估不同模型之间的性能差异，找到更好的模型或进行模型选择
3. Measurement(测量)：
* 用途：用于研究数据集的属性和特性
* 具体含义：测量工具用于对数据集进行分析，探索数据集的结构、分布、偏差等方面的信息
* 示例：数据集大小、样本分布、类别不平衡度等
* 目的：提供对数据集的详细了解，帮助了解数据集的特点和潜在问题

这三种评估工具在Evaluate库中各自独立，用于不同的评估目的。通过使用这些工具，可以全面评估和理解机器学习模型和数据集的表现和特点

In [1]:
import evaluate

In [2]:
# 加载一个评价指标
# 任何度量(metric), 比较(comparison), 和测量(measurement)工具，都使用'evaluate.load'函数加载:

# 下面举一个精度度量例子，首先是加载
#accuracy = evaluate.load("accuracy") # 从网络加载
accuracy = evaluate.load("../evaluate/metrics/accuracy") # 从本地加载

# 输出描述属性：
print(accuracy.description)


Accuracy is the proportion of correct predictions among the total number of cases processed. It can be computed with:
Accuracy = (TP + TN) / (TP + TN + FP + FN)
 Where:
TP: True positive
TN: True negative
FP: False positive
FN: False negative



In [3]:
# 如果你想确保你正在加载正确类型的求值（特别是如果有名称冲突），你可以显式地传递类型：
accuracy = evaluate.load("../evaluate/metrics/accuracy", module_type="metric")
print(accuracy.description)


Accuracy is the proportion of correct predictions among the total number of cases processed. It can be computed with:
Accuracy = (TP + TN) / (TP + TN + FP + FN)
 Where:
TP: True positive
TN: True negative
FP: False positive
FN: False negative



In [4]:
# 你可以看到它描述了理论上 accuracy 是如何工作的。
# 如果你在你的论文中使用这个指标，你想要正确地引用它。你可以查看引文属性：
print(accuracy.citation)


@article{scikit-learn,
  title={Scikit-learn: Machine Learning in {P}ython},
  author={Pedregosa, F. and Varoquaux, G. and Gramfort, A. and Michel, V.
         and Thirion, B. and Grisel, O. and Blondel, M. and Prettenhofer, P.
         and Weiss, R. and Dubourg, V. and Vanderplas, J. and Passos, A. and
         Cournapeau, D. and Brucher, M. and Perrot, M. and Duchesnay, E.},
  journal={Journal of Machine Learning Research},
  volume={12},
  pages={2825--2830},
  year={2011}
}



In [5]:
# 在我们将 accuracy 或其他评价模块应用于用例之前，我们需要知道 accuracy 的输入格式是什么：
print(accuracy.features)

{'predictions': Value('int32'), 'references': Value('int32')}


In [6]:
# 计算评估模块分数的最简单方法是使用必要的输入直接调用compute（）。
# 只需将特征中看到的输入传递给compute（）方法。
# 计算一组数据的accuracy
accuracy.compute(references=[0,1,0,1], predictions=[1,0,0,1])

# 求值模块以字典的形式返回结果。

{'accuracy': 0.5}

In [7]:
# 在某些情况下，您可以迭代地构建预测，在这种情况下使用add()或add_batch()函数。
# 逐个数据对计算
for ref, pred in zip([0,1,0,1], [1,0,0,1]):
    accuracy.add(references=ref, predictions=pred)
accuracy.compute()

{'accuracy': 0.5}

In [8]:
# 按照batch计算
for refs, preds in zip([[0,1],[0,1]], [[1,0],[0,1]]):
    accuracy.add_batch(references=refs, predictions=preds)
accuracy.compute()

{'accuracy': 0.5}

## 组合评估

In [9]:
#clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])
#results = clf_metrics.compute(predictions=[0, 1, 0], references=[0, 1, 1])
#print(results)
clf_metrics = evaluate.combine(["../evaluate/metrics/accuracy", "../evaluate/metrics/f1", "../evaluate/metrics/precision", "../evaluate/metrics/recall"])
results = clf_metrics.compute(predictions=[0, 1, 0], references=[0, 1, 1])
print(results)

{'accuracy': 0.6666666666666666, 'f1': 0.6666666666666666, 'precision': 1.0, 'recall': 0.5}
